<a href="https://colab.research.google.com/github/koojayeong/TextAnalysis/blob/main/Bag_of_Words_Meets_Bags_of_Popcorn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리

# Load Data

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [37]:
# install kaggle 
!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle competitions list

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         92           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        100           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        915           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        162           False  
digit-recognizer                               2030-01-

In [38]:
cd /content/drive/MyDrive/kaggle/BagsOfPopcorn

/content/drive/MyDrive/kaggle/BagsOfPopcorn


In [39]:
!kaggle competitions download -c word2vec-nlp-tutorial

word2vec-nlp-tutorial.zip: Skipping, found more recently modified local copy (use --force to force download)


In [40]:
!unzip word2vec-nlp-tutorial.zip

Archive:  word2vec-nlp-tutorial.zip
  inflating: labeledTrainData.tsv.zip  
  inflating: sampleSubmission.csv    
  inflating: testData.tsv.zip        
  inflating: unlabeledTrainData.tsv.zip  


In [41]:
!unzip labeledTrainData.tsv.zip
!unzip unlabeledTrainData.tsv.zip
!unzip testData.tsv.zip

Archive:  labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    
Archive:  unlabeledTrainData.tsv.zip
  inflating: unlabeledTrainData.tsv  
Archive:  testData.tsv.zip
  inflating: testData.tsv            


# ---------Part 1---------

# Reading the data

In [42]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/labeledTrainData.tsv.zip", 
                    header=0, # the first line of the file contains column names
                    delimiter="\t", # the fields are separated by tabs
                    quoting=3) # ignore doubled quotes

In [43]:
train.shape

(25000, 3)

In [44]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [46]:
#@title 기본 제목 텍스트
print( train["review"][0] )

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

# Data Cleaning and Text Preprocessing

Removing HTML Markup: The BeautifulSoup Package

In [47]:
pip install BeautifulSoup4

In [48]:
from bs4 import BeautifulSoup

example1 = BeautifulSoup(train["review"][0])

print( train["review"][0] )
print( example1.get_text())

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

Dealing with Punctuation, Numbers and Stopwords: NLTK and regular expressions  
  
 * "!!!" or ":-(" 같은 것들은 감정을 담은 words로 취급되어야 한다.  
 * 숫자들은 words로 취급하거나 "NUM" 으로 대체
 *  punctuation and numbers 을 제거할 땐 re 패키지 사용

In [50]:
import re
letters_only = re.sub("[^a-zA-Z]",
                      " ",
                      example1.get_text())
print(letters_only)

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

 tokenization :   
글자들을 소문자로 바꾸고 개별 단어로 split 하기

In [51]:
lower_case = letters_only.lower()
words = lower_case.split()

stopwords :  
큰 의미 없고 자주 등장하는 단어들을 말한다.  
영어에서는 a, is, the 와 같은 조사들  
NLTK 사용

In [56]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [57]:
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [58]:
words = [w for w in words if not w in stopwords.words("english")]
print(words)

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

u'towards', u'press', u'also' 같은 문자열 앞 "u"는 unicode string으로서 나타내는 것을 말한다.

Porter Stemming, Lemmatizing 등 다르게 표현된 같은 단어들을 처리하는 방법이 있다.  
"messages", "message", "messaging"

In [66]:
def review_to_words( raw_review) :
  """
  raw review를 string of words 로 바꿔주기 위한 함수
  """
  # 1. remove html
  review_text = BeautifulSoup(raw_review).get_text()

  # 2. remove non-letters
  letters_only = re.sub("[^a-zA-Z]", 
                        " ",
                        review_text)
  
  # 3. convert to lower case, split into individual words
  words = letters_only.lower().split()

  # 4. python. searching a set is much faster than searching
  # a list, so convert the stop words to a set
  stops = set(stopwords.words("english"))

  # 5. remove stop words
  meaningful_words = [w for w in words if not w in stops]

  # 6. one string 으로 반환
  return " ".join(meaningful_words)

data type 으로 set을 이용하면 속도가 빠르다. 

In [67]:
clean_review = review_to_words(train["review"][0])
print(clean_review)

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [70]:
# 진행 상황 확인

print("Cleaning and parsing the training set movie reviews...\n")

num_reviews = train["review"].size

clean_train_reviews = []

for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_reviews ))                                                                    
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



# Creating Features from a Bag of Words(using scikit-learn)

IMDB 데이터를 bag of word 표현법으로 할 것이다. 가장 빈도수가 높은 5000개의 단어를 활용할 것이다. 

In [72]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000)

train_data_features = vectorizer.fit_transform(clean_train_reviews)

train_data_features = train_data_features.toarray()

In [73]:
print(train_data_features.shape)

(25000, 5000)


In [74]:
vocab = vectorizer.get_feature_names()
print(vocab)

['abandoned', 'abc', 'abilities', 'ability', 'able', 'abraham', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accompanied', 'accomplished', 'according', 'account', 'accuracy', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'activities', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'ad', 'adam', 'adams', 'adaptation', 'adaptations', 'adapted', 'add', 'added', 'adding', 'addition', 'adds', 'adequate', 'admire', 'admit', 'admittedly', 'adorable', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventure', 'adventures', 'advertising', 'advice', 'advise', 'affair', 'affect', 'affected', 'afford', 'aforementioned', 'afraid', 'africa', 'african', 'afternoon', 'afterwards', 'age', 'aged', 'agent', 'agents', 'ages', 'aging', 'ago', 'ag

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [75]:
import numpy as np

dist = np.sum(train_data_features, axis=0)

for tag, count in zip(vocab, dist) :
  print(count, tag)

187 abandoned
125 abc
108 abilities
454 ability
1259 able
85 abraham
116 absence
83 absent
352 absolute
1485 absolutely
306 absurd
192 abuse
91 abusive
98 abysmal
297 academy
485 accent
203 accents
300 accept
130 acceptable
144 accepted
92 access
318 accident
200 accidentally
88 accompanied
124 accomplished
296 according
186 account
81 accuracy
284 accurate
123 accused
179 achieve
139 achieved
124 achievement
90 acid
971 across
1251 act
658 acted
6490 acting
3354 action
311 actions
83 activities
2389 actor
4486 actors
1219 actress
369 actresses
394 acts
793 actual
4237 actually
148 ad
302 adam
98 adams
453 adaptation
80 adaptations
154 adapted
810 add
439 added
166 adding
347 addition
337 adds
113 adequate
124 admire
621 admit
134 admittedly
101 adorable
510 adult
376 adults
100 advance
90 advanced
153 advantage
510 adventure
204 adventures
91 advertising
259 advice
90 advise
346 affair
93 affect
113 affected
104 afford
126 aforementioned
343 afraid
212 africa
255 african
187 afternoon

# Random Forest

In [76]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators= 100)

forest = forest.fit(train_data_features, train["sentiment"])

Training the random forest...


In [78]:
test = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/testData.tsv",
                   header = 0,
                   delimiter= '\t',
                   quoting = 3)

print(test.shape)

num_reviews = len(test["review"])
clean_test_reviews = []

print("Cleaning and parsing the test set movie reviews...\n")

for i in range(0, num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = forest.predict(test_data_features)

output = pd.DataFrame(data = {"id": test["id"], "sentiment": result})

output.to_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/Bag_of_Words_model.csv",
              index=False,
              quoting=3)

(25000, 2)
Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [84]:
# kaggle api 로 제출하기
!kaggle competitions submit -c word2vec-nlp-tutorial -f /content/drive/MyDrive/kaggle/BagsOfPopcorn/Bag_of_Words_model.csv -m "21.12.28"

100% 276k/276k [00:01<00:00, 172kB/s]
Successfully submitted to Bag of Words Meets Bags of Popcorn

# ---------Part 2---------

# Word2Vec : Distributed Word Vectors

gensim 패키지를 이용해 Word2Vec 구현 사용.   
  
word2vec은 컴퓨팅 집약적이므로 멀티스레딩에 의존한다. cython을 설치해야 적절한 시간 내에 모델 훈련할 수 있다.

In [86]:
import sys
import cython

import numba
from numba import jit

import numpy as np

In [85]:
pip install gensim

# Preparing to Train a Model

In [87]:
import pandas as pd

# read data from files
train = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/labeledTrainData.tsv",
                    header=0,
                    delimiter='\t',
                    quoting=3)
test = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/testData.tsv",
                    header=0,
                    delimiter='\t',
                    quoting=3)
unlabeled_train = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/unlabeledTrainData.tsv",
                    header=0,
                    delimiter='\t',
                    quoting=3)

print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))


Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [88]:
# string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False) :
  # 1. remove html
  review_text = BeautifulSoup(review).get_text()

  # 2. remove non-letters
  review_text = re.sub("[^a-zA-Z]", " ", review_text)

  # 3. 소문자화, split
  words = review_text.lower().split()

  # 4. 선택사항 : stopwords
  if remove_stopwords :
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]

  return words

word2voec 의 입력 형식은 list의 lists 이다. 문단을 문장으로 분리해야 하는데 "?", "!", "." 으로 끝나거나 공백, 대문자 등 명확한 하나의 기준은 없다.  
   
문장 분할을 위해 nltk의 토큰나이저를 사용할 것이다.

In [98]:
import nltk
nltk.download('punkt')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences(review, tokenizer, remove_stopwords=False) :
  """
  review를 parased sentences로 split한다.
  list of sentences 를 반환한다.
  """
  # 1. use nltk tokenizer to split paragraph into sentences
  raw_sentences = tokenizer.tokenize(review.strip())

  # 2. loop over each sentence
  sentences = []
  for raw_sentence in raw_sentences :
    # 만약 sentence 가 비었다면 skip it.
    if len(raw_sentence) > 0 :
      sentences.append(review_to_wordlist(raw_sentence,
                                          remove_stopwords))
  return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [99]:
sentences = []

print("Parsing sentences from training set")
for review in train["review"]:
  sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
  sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

In [100]:
print(len(sentences))

print(sentences[0])

print(sentences[1])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


# Training and Saving Model

* Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.
* Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
* Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
* Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.
* Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
* Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
* Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

In [101]:
# output 메세지 format
import logging 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level = logging.INFO)

# parameter 값 setting
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# model 초기화, train
from gensim.models import word2vec
print("Training moel...")
model = word2vec.Word2Vec(sentences,
                          workers=num_workers,
                          size=num_features,
                          min_count=min_word_count,
                          window=context,
                          sample=downsampling)

# model이 더 학습되지 않는다면 calling
# init_sims가 모델을 더 memory 효율적으로 만든다
model.init_sims(replace=True)

model_name="300features_40minwords_10context"
model.save(model_name)

2021-12-28 09:20:28,314 : INFO : collecting all words and their counts
2021-12-28 09:20:28,320 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-12-28 09:20:28,420 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types


Training moel...


2021-12-28 09:20:28,562 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2021-12-28 09:20:28,667 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2021-12-28 09:20:28,772 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2021-12-28 09:20:28,919 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2021-12-28 09:20:29,109 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2021-12-28 09:20:29,216 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2021-12-28 09:20:29,361 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2021-12-28 09:20:29,471 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2021-12-28 09:20:29,588 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50

In [102]:
directory = "/content/drive/MyDrive/kaggle/BagsOfPopcorn/"
model_name="300features_40minwords_10context"
model.save(directory+model_name)

2021-12-28 09:26:36,489 : INFO : saving Word2Vec object under /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context, separately None
2021-12-28 09:26:36,494 : INFO : not storing attribute vectors_norm
2021-12-28 09:26:36,497 : INFO : not storing attribute cum_table
2021-12-28 09:26:37,006 : INFO : saved /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context


In [104]:
!top -o cpu

top: unrecognized field name 'cpu'


# Exploring the Model Results

"doesn_match" 함수는 집합에서 어떤 단어가 다른 단어와 가장 다른지 추론한다.

In [105]:
model.doesnt_match("man woman child kitchen".split())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [106]:
model.doesnt_match("france england germany berlin".split())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'berlin'

In [107]:
model.doesnt_match("paris berlin london austria".split())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'paris'

또한 "most_simply" 함수를 사용하여 모형의 단어 군집에 대한 통찰력을 얻을 수 있습니다.

In [108]:
model.most_similar("man")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6262633800506592),
 ('lady', 0.5831054449081421),
 ('lad', 0.5641592741012573),
 ('monk', 0.540630042552948),
 ('soldier', 0.5242375135421753),
 ('guy', 0.5180153846740723),
 ('farmer', 0.514233410358429),
 ('men', 0.5024681091308594),
 ('businessman', 0.4997982978820801),
 ('chap', 0.49405404925346375)]

In [109]:
model.most_similar("queen")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('princess', 0.664909839630127),
 ('mistress', 0.616767406463623),
 ('maid', 0.6099587678909302),
 ('bride', 0.6042027473449707),
 ('showgirl', 0.5805788636207581),
 ('belle', 0.5802614688873291),
 ('victoria', 0.5770310163497925),
 ('madame', 0.5767858028411865),
 ('latifah', 0.5751155018806458),
 ('goddess', 0.5747115612030029)]

In [110]:
model.most_similar("awful")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7571889162063599),
 ('horrible', 0.7385010719299316),
 ('atrocious', 0.7222769260406494),
 ('dreadful', 0.7118343114852905),
 ('abysmal', 0.7115039229393005),
 ('horrendous', 0.6855192184448242),
 ('horrid', 0.6493418216705322),
 ('appalling', 0.6374698877334595),
 ('lousy', 0.6248220205307007),
 ('laughable', 0.6199133396148682)]

# ---------Part 3---------

# Numeric Representations of Words

trained model에서 단어의 의미적인 이해를 하기 위해 무엇을 사용해야 할까?   
"syn0" 이라 불리는 numpy에 vocabulary의 각 단어의 feature vector가 저장되어 있다. 

In [112]:
from gensim.models import Word2Vec

directory = "/content/drive/MyDrive/kaggle/BagsOfPopcorn/"
model_name="300features_40minwords_10context"

model = Word2Vec.load(directory+model_name)

print(type(model.syn0))
print(model.syn0.shape)

2021-12-28 09:57:17,975 : INFO : loading Word2Vec object from /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context
2021-12-28 09:57:18,559 : INFO : loading wv recursively from /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context.wv.* with mmap=None
2021-12-28 09:57:18,565 : INFO : setting ignored attribute vectors_norm to None
2021-12-28 09:57:18,567 : INFO : loading vocabulary recursively from /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context.vocabulary.* with mmap=None
2021-12-28 09:57:18,572 : INFO : loading trainables recursively from /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context.trainables.* with mmap=None
2021-12-28 09:57:18,573 : INFO : setting ignored attribute cum_table to None
2021-12-28 09:57:18,576 : INFO : loaded /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context


AttributeError: ignored

# From Words To Paragraphs, Attempt 1 : Vector Averaging

# From Words to Paragraphs, Attempt 2 : Clustering